<a href="https://colab.research.google.com/github/swapanjaliVeccProject/Sign-Language-Recognition-Neural-Network/blob/main/Sign_Language_Recognition_Data_Creation_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Libraries

In [ ]:
!pip install mediapipe

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import math
import numpy as np
import os 
import glob
import mediapipe as mp
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import csv
import pandas as pd
import math

# Downloading & Unzipping the Dataset

In [ ]:
!wget  http://www.cvssp.org/FingerSpellingKinect2011/fingerspelling5.tar.bz2

In [ ]:
!tar -xvf /content/fingerspelling5.tar.bz2      #[run this cell to extract tar.bz2 files]

# Selecting `n` images from the directory and resizing to 64px while maintaining Aspect Ratio

In [ ]:
imagelist=[]

ctr = 1
for dir in os.listdir('/content/dataset5'):
  path='/content/dataset5/'+str(dir)
  for subdir in os.listdir(path):
    subpath=path+'/'+str(subdir)
    for name in glob.glob(subpath+'/color_?*'):
      if ctr <= 64: #n
        imagelist.append(name)
        ctr += 1
    ctr=1;

DESIRED_HEIGHT = 64
DESIRED_WIDTH = 64
def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2.imwrite()


# Read images with OpenCV.
images = {name: cv2.imread(name) for name in imagelist}
# Preview the images.
for name, image in images.items():
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2.imwrite(name,img)

In [ ]:
print(len(imagelist))
print(len(images))

In [ ]:
for name, image in images.items():
  h, w = image.shape[:2]
  print(h,w)

# Using Mediapipe to Annotate the 21 Hand Landmarks

## Saving the coordinates in a CSV File

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
plt.imshow(cv2.flip(image[:,:,::-1],1));plt.show()

In [ ]:
mp_hands = mp.solutions.hands
model=mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.2)

for name, image in images.items():
  results = model.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
  print('Handedness:', results.multi_handedness)
  # print(name)
  if not results.multi_hand_landmarks:
    # print("no hand detected")
    continue
  image_height, image_width, _ = image.shape
  annotated_image = cv2.flip(image.copy(),1)

  new_list = []
  for hand_landmarks in results.multi_hand_landmarks:
    for point in mp_hands.HandLandmark:
      print(point, point.value)
      if point.value in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
        normalizedLandmark = hand_landmarks.landmark[point]
        pixelCoordinatesLandmark = mp_drawing._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, image_width, image_height)
        cv2.circle(annotated_image, pixelCoordinatesLandmark, 1, (0, 255, 0), -1)
        if point.value == 20:
          cv2_imshow(annotated_image)

        # print('hand_landmarks:', normalizedLandmark)
        new_list.append([name,point,point.value,normalizedLandmark.x,normalizedLandmark.y,normalizedLandmark.z])

  with open('Detected_Image_Data_64.csv','a') as f:
    write = csv.writer(f)
    write.writerows(new_list)

# Finding the Angles between the Adjacent Landmarks

## Saving the Data in a CSV File

In [ ]:
df = pd.read_csv('/content/Detected_Image_Data_64.csv')
df.columns=['name', 'landmark', 'points', 'x', 'y', 'z']
print(df.iloc[4321,])

In [ ]:
adjacent_nodes = [(0,1),(1,2),(2,3),(3,4),(0,5),(5,6),(6,7),(7,8),(5,9),(9,10),(10,11),(11,12),(9,13),(13,14),(14,15),(15,16),(17,18),(18,19),(19,20),(0,17),(13,17)]
adj_list=[]
for item in adjacent_nodes:
  temp_list=[]
  temp_list=adjacent_nodes.copy()
  temp_list.remove(item)
  for rem in temp_list:
    if((item[1]==rem[0] and item[0]!=rem[1]) or (item[0]==rem[0] and item[1]!=rem[1]) or (item[1]==rem[1] and item[0]!=rem[0])):
      newT=((item[0], item[1]), (rem[0], rem[1]))
      adj_list.append(newT)

#unique adjacent lists
for a in adj_list:
  temp_list=[]
  temp_list=adj_list.copy()
  temp_list.remove(a)
  for b in temp_list:
    if a[0]==b[1] and a[1]==b[0]:
      adj_list.remove(b)

In [ ]:
for i in adj_list:
  print(i)

print(len(adj_list))

In [ ]:
name_list=[]
for ind in range(0, df.shape[0]-21, 21):
  name_list.append(df.iloc[ind, 0])

In [ ]:
len(name_list)

In [ ]:
ang_list=[]
angle_list=[]
final_list=[]

# Calculating the Angles between the nodes
def calculateAngle(x1, y1, z1,x2, y2, z2,x3, y3, z3):   
  # Find direction ratio of line AB
  ABx = x1 - x2;
  ABy = y1 - y2;
  ABz = z1 - z2;
  # Find direction ratio of line BC
  BCx = x3 - x2;
  BCy = y3 - y2;
  BCz = z3 - z2;
  # Find the dotProduct
  # of lines AB & BC
  dotProduct = (ABx * BCx + ABy * BCy + ABz * BCz);
  # Find magnitude of
  # line AB and BC
  magnitudeAB = (ABx * ABx + ABy * ABy + ABz * ABz);
  magnitudeBC = (BCx * BCx + BCy * BCy + BCz * BCz);
  # Find the cosine of
  # the angle formed
  # by line AB and BC
  angle = dotProduct;
  angle /= math.sqrt(magnitudeAB * magnitudeBC);
  # Find angle in radian
  # angle = (angle * 180) / 3.14;
  return (np.arccos(angle)*180)/3.14


for i in range(0,df.shape[0]-21,21):
  temp_df=df.iloc[i:i+21,:]
  for a in adj_list:

    first=a[0][0]
    second=a[0][1]
    third=a[1][0]
    fourth=a[1][1]
    x1=temp_df.iloc[first,:].x
    y1=temp_df.iloc[first,:].y
    z1=temp_df.iloc[first,:].z

    x2=temp_df.iloc[second,:].x
    y2=temp_df.iloc[second,:].y
    z2=temp_df.iloc[second,:].z

    x3=temp_df.iloc[third,:].x
    y3=temp_df.iloc[third,:].y
    z3=temp_df.iloc[third,:].z

    x4=temp_df.iloc[fourth,:].x
    y4=temp_df.iloc[fourth,:].y
    z4=temp_df.iloc[fourth,:].z

# Finding the common points and then calculating the angles
    if(second == third):
      angle=calculateAngle(x1, y1, z1, x2, y2, z2, x4, y4, z4)
    elif(first == third):
      angle=calculateAngle(x3, y3, z3, x2, y2, z2, x4, y4, z4)
    elif(second == fourth):
      angle=calculateAngle(x1, y1, z1, x4, y4, z4, x3, y3, z3)

    ang_list.append(round(angle, 4))

  angle_list.append(ang_list)
  ang_list = []

print('Angle',len(angle_list))
print('Name',len(name_list))

for i in range(0, len(name_list)):
  final_list.append([name_list[i], angle_list[i]])

with open('Angles_Images.csv','a') as f:
    write = csv.writer(f)
    write.writerows(final_list)

In [ ]:
print(len(final_list))
print(len(angle_list))

In [ ]:
print(final_list[0])
# print(len(final_list[0][1]))

In [ ]:
for i in angle_list:
  print(i)

print(len(angle_list))

In [ ]:
for item in angle_list:
  print(len(set(item)))